In [1]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
df = sns.load_dataset('penguins')
dfcopia = df.copy()
dfcopia = dfcopia.dropna(axis=0)
especies = dfcopia['species'].unique()
print(especies)

dfcopia['specie numerica'] = 0

i = 0
for pinguin in dfcopia['species']:
    #print(pinguin)
    if pinguin == 'Adelie':
        dfcopia['specie numerica'].iloc[i] = 1
        i += 1
    elif pinguin == 'Chinstrap':
        dfcopia['specie numerica'].iloc[i] = 2
        i += 1
    elif pinguin == 'Gentoo':
        dfcopia['specie numerica'].iloc[i] = 3
        i += 1
    #print(dfcopia['species'].iloc[i-1],dfcopia['specie numerica'].iloc[i-1])

features_numericos = ["bill_length_mm","bill_depth_mm", "flipper_length_mm", "body_mass_g"]
df_numericos = dfcopia[features_numericos]

['Adelie' 'Chinstrap' 'Gentoo']


c:\venv\ilumpy\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
sc = StandardScaler()
sc.fit(df_numericos)
X_norm = sc.transform(df_numericos)

# Testando Modelos Diferentes
Testaremos a validação cruzada com o modelo de floresta aleatória.
Para esse teste, modifiquei 3 dos hiperparâmetros da floresta aleatória: *Número de folhas*, *Número de árvores* e o *Número de profundidade*. Para cada um dos hiperparâmetros, escolhi 3 números. Dessa forma, o número total de modelos diferentes é igual a 3³, ou 27. Também estipulei o número de k-fold para 15

In [4]:
# ----- Carregando os dados e definindo parâmetros:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]
TARGET = ["specie numerica"]

df = dfcopia

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size = TAMANHO_TESTE, random_state = SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

# o .values utiliza apenas os valores, em forma de array
x_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values
x_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [5]:
# ----- Importando bibliotecas necessárias
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [17]:
# ----- Aqui o score estipulado é o R².
from itertools import product
from sklearn.metrics import r2_score

NUM_ARVORES = [50, 100, 1000]
NUM_FOLHAS = [10, 15, 30]
NUM_PROFUNDIDADE = [5, 10, 15]
NUM_FOLDS = 15
NUM_CPU_CORES = 4

for n_folhas, n_profundidade, n_arvores in product(
    NUM_FOLHAS, NUM_PROFUNDIDADE, NUM_ARVORES
):
    modelo_rf = RandomForestRegressor(
        n_estimators=n_arvores,
        max_leaf_nodes=n_folhas,
        max_depth=n_profundidade,
        random_state=SEMENTE_ALEATORIA,
        n_jobs= NUM_CPU_CORES,
    )
    
    scores = cross_val_score(
    #modelo_rf.fit(x_treino, y_treino)
    modelo_rf,
    x_treino,
    y_treino.ravel(),
    cv=NUM_FOLDS,
    )

    print(
        f"Aplicando a métrica de erro R² para os seguintes hiperparâmetros: \n"
        f"{n_arvores} número de árvores,\n"
        f"{n_folhas} número de folhas e\n"
        f"{n_profundidade} número de profundidade\n"
        f"temos que:\n"
        f"Os scores foram de: {scores}\n"
        f"A média dos scores é de: {scores.mean()}"
        f"\n"
    )

Aplicando a métrica de erro R² para os seguintes hiperparâmetros: 
50 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [0.99997102 0.93950786 0.99724105 0.9782757  0.96864125 0.99692537
 0.99999975 0.99059597 0.96873463 0.99615739 0.93284281 0.99423925
 0.95221341 0.98808175 0.93522041]
A média dos scores é de: 0.9759098419911065

Aplicando a métrica de erro R² para os seguintes hiperparâmetros: 
100 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [0.99998298 0.93827879 0.99759013 0.96693982 0.96084947 0.99773394
 0.99997733 0.98750996 0.96835683 0.996556   0.93365366 0.99423732
 0.96440144 0.98833949 0.93177798]
A média dos scores é de: 0.975079008517195

Aplicando a métrica de erro R² para os seguintes hiperparâmetros: 
1000 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [0.99999919 0.93461165 0.99540958 0.97087671 0.96599957 0.99756373
 0

In [15]:
# ----- Aqui o score estipulado é o RMSE
NUM_ARVORES = [10, 50, 1000]
NUM_FOLHAS = [10, 15, 30]
NUM_PROFUNDIDADE = [5, 10, 15]
NUM_FOLDS = 15
NUM_CPU_CORES = 4

for n_folhas, n_profundidade, n_arvores in product(
    NUM_FOLHAS, NUM_PROFUNDIDADE, NUM_ARVORES
):
    modelo_rf = RandomForestRegressor(
        n_estimators=n_arvores,
        max_leaf_nodes=n_folhas,
        max_depth=n_profundidade,
        random_state=SEMENTE_ALEATORIA,
        n_jobs= NUM_CPU_CORES,
    )
    
    scores = cross_val_score(
    #modelo_rf.fit(x_treino, y_treino)
    modelo_rf,
    x_treino,
    y_treino.ravel(), ## <----------------------------- Sem o .ravel um erro aparece.
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
    )

    print(
        f"Aplicando a métrica de erro RMSE para os seguintes hiperparâmetros: \n"
        f"{n_arvores} número de árvores,\n"
        f"{n_folhas} número de folhas e\n"
        f"{n_profundidade} número de profundidade\n"
        f"temos que:\n"
        f"Os scores foram de: {scores}\n"
        f"A média dos scores é de: {scores.mean()}"
        f"\n"
    )

Aplicando a métrica de erro RMSE para os seguintes hiperparâmetros: 
10 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [-0.02237601 -0.2236068  -0.07130164 -0.13027308 -0.11401754 -0.06761863
 -0.00062619 -0.08065543 -0.17175564 -0.06992059 -0.2236068  -0.09350066
 -0.17897326 -0.13691718 -0.18353259]
A média dos scores é de: -0.11791213484805145

Aplicando a métrica de erro RMSE para os seguintes hiperparâmetros: 
50 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [-0.00469345 -0.22808605 -0.03434312 -0.11974155 -0.13132901 -0.05340153
 -0.00046737 -0.08605622 -0.15081629 -0.05287245 -0.22405357 -0.07240359
 -0.19398953 -0.08869072 -0.21682839]
A média dos scores é de: -0.11051818925329199

Aplicando a métrica de erro RMSE para os seguintes hiperparâmetros: 
1000 número de árvores,
10 número de folhas e
5 número de profundidade
temos que:
Os scores foram de: [-0.00078683 -0.23713705 -0.0

# Afinal, apresentou Overfit ou não? 

Nos 27 modelos testados com k-fold, percebemos que não houve de fato overfit, já que os diferentes modelos manteram o resultado médio dentro de uma margem de erro muito próxima. Em R², o score de todos os testes deu um resultado muito bom de aproximadamente 0.97. Em RMSE, também teve um valor muito bom de aprozimadamente 0.11. Inclusive, o melhor modelo teve um dado bizarro, pois uma das predições foi perfeita. Mas, ainda assim, aparenta ter sido um caso de coincidencia e não de overfitting, já que a média dos scores continuou dentro do padrão.

Ademais, por causa desse resultado muito bom, acreditamos que o modelo de árvore de decisões é de fato um ótimo modelo de predições taxonômicas e diferenciação de espécie com base em dados de características físicas, que foi o nosso trabalho.